In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2

# Try 1

In [ ]:
page_x_inches: float = 17. # inches
page_y_inches: float = 11 # inches
border:float = 20.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
all_circles = []
circle_to_fill = drawbox
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.45, 0.08)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 20)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 1
iter_max = 10
rad_mults = (0.7, 0.06)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
bottom_circles = MultiPolygon([c for c in all_circles if not getattr(c, 'filled', False)])
bottom_circles

In [ ]:
all_circles = []
circle_to_fill = drawbox
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.15, 0.08)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 10)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 1
iter_max = 5
rad_mults = (0.25, 0.25)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
tri_vertices = MultiPoint([c.centroid for c in all_circles if not getattr(c, 'filled', False)])

In [ ]:
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

In [ ]:
length_filters = np.random.uniform(96, 185, size=len(trils))
tri_buffer = 2.3
ftrils = MultiLineString([l for l, lf in zip(trils, length_filters) if l.length<lf])
tris = ftrils.buffer(tri_buffer, cap_style=2, join_style=3)
tris

In [ ]:
stp = gp.ScaleTransPrms(
    d_buffer=-0.3, 
    angles=-60,

)
tri_fill = gp.scale_trans(tris, **stp.prms)

In [ ]:
tri_layer = gp.merge_LineStrings([p.boundary for p in tri_fill])

In [ ]:
layer = tri_layer.intersection(bottom_circles)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(1)
sk.geometry(layer)
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0108_waka_waka.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.3mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

# try 2

In [ ]:
page_x_inches: float = 17. # inches
page_y_inches: float = 11 # inches
border:float = 20.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
all_circles = []
circle_to_fill = drawbox
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.48, 0.05)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 30)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 1
iter_max = 10
rad_mults = (0.7, 0.05)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
bottom_circles = MultiPolygon([c for c in all_circles if not getattr(c, 'filled', False)])
bottom_circles

In [ ]:
all_circles = []
circle_to_fill = drawbox.buffer(10)
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.15, 0.08)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 10)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 5
iter_max = 15
rad_mults = (0.3, 0.2)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
tri_vertices = MultiPoint([c.centroid for c in all_circles if not getattr(c, 'filled', False)])

trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

length_filters = np.random.uniform(96, 185, size=len(trils))
ftrils = MultiLineString([l for l, lf in zip(trils, length_filters) if l.length<lf])
tris = [f.buffer(np.random.uniform(1, 2.5), cap_style=2, join_style=3) for f in ftrils]
tris = gp.Poly(so.unary_union(tris))

stp = gp.ScaleTransPrms(
    d_buffer=-0.25, 
    angles=-60,

)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
tris.fill_scale_trans(**stp.prms)
tri_lattice1 = tris

In [ ]:
all_circles = []
circle_to_fill = drawbox.buffer(10)
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.15, 0.08)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 10)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 5
iter_max = 25
rad_mults = (0.18, 0.18)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
tri_vertices = MultiPoint([c.centroid for c in all_circles if not getattr(c, 'filled', False)])

trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

length_filters = np.random.uniform(96, 185, size=len(trils))
ftrils = MultiLineString([l for l, lf in zip(trils, length_filters) if l.length<lf])
tris = [f.buffer(np.random.uniform(1, 2.2), cap_style=2, join_style=3) for f in ftrils]
tris = gp.Poly(so.unary_union(tris))

stp = gp.ScaleTransPrms(
    d_buffer=-0.35, 
    angles=-60,

)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
tris.fill_scale_trans(**stp.prms)
tri_lattice2 = tris

In [ ]:
layer1 = tri_lattice1.fill.intersection(bottom_circles)
layer2 = tri_lattice2.fill.difference(tri_lattice1.p).intersection(bottom_circles)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(1)
sk.geometry(layer1)
sk.stroke(2)
sk.geometry(layer2)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0118_waka_waka.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.1mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

# try 3

In [ ]:
page_x_inches: float = 17. # inches
page_y_inches: float = 11 # inches
border:float = 20.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
all_circles = []
circle_to_fill = drawbox
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.48, 0.05)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 30)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 2
iter_max = 50
rad_mults = (0.7, 0.05)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
bottom_circles = MultiPolygon([c for c in all_circles if not getattr(c, 'filled', False)])
bottom_circles

In [ ]:
all_circles = []
circle_to_fill = drawbox.buffer(10)
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.15, 0.08)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 10)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 5
iter_max = 15
rad_mults = (0.3, 0.2)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
tri_vertices = MultiPoint([c.centroid for c in all_circles if not getattr(c, 'filled', False)])

trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

length_filters = np.random.uniform(96, 185, size=len(trils))
ftrils = MultiLineString([l for l, lf in zip(trils, length_filters) if l.length<lf])
tris = [f.buffer(np.random.uniform(1.2, 3), cap_style=2, join_style=3) for f in ftrils]
tris = gp.Poly(so.unary_union(tris))

stp = gp.ScaleTransPrms(
    d_buffer=-0.22, 
    angles=-60,

)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
tris.fill_scale_trans(**stp.prms)
tri_lattice1 = tris

In [ ]:
all_circles = []
circle_to_fill = drawbox.buffer(10)
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.15, 0.08)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 10)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 5
iter_max = 15
rad_mults = (0.18, 0.18)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
tri_vertices = MultiPoint([c.centroid for c in all_circles if not getattr(c, 'filled', False)])

trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

length_filters = np.random.uniform(96, 185, size=len(trils))
ftrils = MultiLineString([l for l, lf in zip(trils, length_filters) if l.length<lf])
tris = [f.buffer(np.random.uniform(1, 2.5), cap_style=2, join_style=3) for f in ftrils]
tris = gp.Poly(so.unary_union(tris))

stp = gp.ScaleTransPrms(
    d_buffer=-0.25, 
    angles=-60,

)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
tris.fill_scale_trans(**stp.prms)
tri_lattice2 = tris

In [ ]:
all_circles = []
circle_to_fill = drawbox.buffer(10)
# init
fc_rad = gp.get_rad(circle_to_fill)
x = circle_to_fill.centroid.x
rad_mults = (0.15, 0.08)
rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 10)
circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
all_circles += list(circles)
circle_to_fill.filled = True
not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
areas = np.array([c.area for c in not_yet_filled])
areas = areas/areas.sum()
circle_to_fill = np.random.choice(not_yet_filled, p=areas)
circles

In [ ]:
min_rad = 5
iter_max = 20
rad_mults = (0.18, 0.18)

In [ ]:
n_fails = 0
n_iters = 0
pbar = tqdm(total=iter_max)
while (n_fails < 20) and (n_iters < iter_max):
    try:
#         pbar.update()
        n_iters += 1
        fc_rad = gp.get_rad(circle_to_fill)
        
        rads = np.linspace(fc_rad*rad_mults[0], fc_rad*rad_mults[1], 80)
        rads[rads<min_rad] = min_rad
        circles = gp.circle_pack_within_poly(circle_to_fill, rads, progress_bar=False)
        circle_to_fill.filled = True
        all_circles += list(circles)
        not_yet_filled = [c for c in all_circles if not getattr(c, 'filled', False)]
        areas = np.array([c.area for c in not_yet_filled])
        areas = areas/areas.sum()
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        n_fails = 0
    except:
        n_fails += 1
        circle_to_fill = np.random.choice(not_yet_filled, p=areas)
        
print(f'total iters = {n_iters}')
print(f'total fails = {n_fails}')
MultiPolygon(all_circles).boundary

In [ ]:
tri_vertices = MultiPoint([c.centroid for c in all_circles if not getattr(c, 'filled', False)])

trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

length_filters = np.random.uniform(96, 185, size=len(trils))
ftrils = MultiLineString([l for l, lf in zip(trils, length_filters) if l.length<lf])
tris = [f.buffer(np.random.uniform(0.8, 1.8), cap_style=2, join_style=3) for f in ftrils]
tris = gp.Poly(so.unary_union(tris))

stp = gp.ScaleTransPrms(
    d_buffer=-0.35, 
    angles=-60,

)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
tris.fill_scale_trans(**stp.prms)
tri_lattice3 = tris

In [ ]:
layer1 = tri_lattice1.fill.intersection(bottom_circles)
layer2 = tri_lattice2.fill.difference(tri_lattice1.p).intersection(bottom_circles)
layer3 = tri_lattice3.fill.difference(tri_lattice1.p).difference(tri_lattice2.p).intersection(bottom_circles)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(1)
sk.geometry(layer1)
sk.stroke(2)
sk.geometry(layer2)
sk.stroke(3)
sk.geometry(layer3)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0119_waka_waka.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.25mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

# Try 4

In [ ]:
page_x_inches: float = 17. # inches
page_y_inches: float = 11 # inches
border:float = 20.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.48,
    rad_seq_end=0.05,
    min_allowed_rad=2,
    n_rads=30,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.6
rcp.rad_seq_end=0.04
rcp.n_rads=80
rcp.run(50, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(2.1, 2.8).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
occluded_tris = tris.intersection(rcp.unfilled_circles).buffer(-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.4, 2.2).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris]).buffer(-1e-6)

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(10), rad_seq_start=0.15,
    rad_seq_end=0.15, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.2
trcp.rad_seq_end=0.2
trcp.run(4)

In [ ]:
length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.5, 1.8).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = gp.merge_Polygons([all_polys, occluded_tris])

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(1)
sk.geometry(layer1)
sk.stroke(2)
sk.geometry(layer2)
sk.stroke(3)
sk.geometry(layer3)
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0120_waka_waka.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.1mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

# Try 5

In [ ]:
page_x_inches: float = 17. # inches
page_y_inches: float = 11 # inches
border:float = 20.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.48,
    rad_seq_end=0.05,
    min_allowed_rad=2,
    n_rads=30,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.7
rcp.rad_seq_end=0.09
rcp.n_rads=80
rcp.run(10, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.12,
    rad_seq_end=0.12, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

In [ ]:

def map_on_xrange(ls, xp, fp, *args, **kwargs):
    return np.interp(ls.centroid.x, xp=xp, fp=fp)
xp = [drawbox.bounds[0], drawbox.bounds[2]]

In [ ]:
vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])

In [ ]:
DL = gp.DelauneyLattice(
    vertices=vertices,
    max_length_filter_gen=lambda *args: ss.uniform(96, 185).rvs(),
    min_length_filter_gen=gp.make_callable(0),
    buffer_size_gen=functools.partial(map_on_xrange, xp=xp, fp=[0.8, 2.5]),
    buffer_individually=True,
)

In [ ]:
occluded_tris = so.unary_union(DL.polys).intersection(rcp.unfilled_circles).buffer(1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.35,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.10,
    rad_seq_end=0.10, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)


In [ ]:
vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])

In [ ]:
DL = gp.DelauneyLattice(
    vertices=vertices,
    max_length_filter_gen=lambda *args: ss.uniform(96, 185).rvs(),
    min_length_filter_gen=gp.make_callable(0),
    buffer_size_gen=functools.partial(map_on_xrange, xp=xp, fp=[2.5, 0.8]),
    buffer_individually=True,
)

In [ ]:
int_tris = so.unary_union(DL.polys).intersection(rcp.unfilled_circles).buffer(-1e-6)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.35,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris]).buffer(1e-6)

In [ ]:
all_polys

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)


In [ ]:
vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])

In [ ]:
DL = DelauneyLattice(
    vertices=vertices,
    max_length_filter_gen=lambda *args: ss.uniform(96, 185).rvs(),
    min_length_filter_gen=gp.make_callable(0),
    buffer_size_gen=functools.partial(map_on_xrange, xp=xp, fp=[0.5, 2]),
    buffer_individually=True,
)

In [ ]:
int_tris = so.unary_union(DL.polys).intersection(rcp.unfilled_circles).buffer(1e-6)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.35,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris]).buffer(1e-6)

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(1)
sk.geometry(layer1)
sk.stroke(2)
sk.geometry(layer2)
sk.stroke(3)
sk.geometry(layer3)
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0120_waka_waka.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.1mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

# Try 6

In [ ]:
page_x_inches: float = 17. # inches
page_y_inches: float = 11 # inches
border:float = 20.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.5,
    rad_seq_end=0.1,
    min_allowed_rad=2,
    n_rads=90,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.5
rcp.rad_seq_end=0.1
rcp.n_rads=80
rcp.run(3, progress_bar=True)
rcp.unfilled_circles

In [ ]:
# init
rcp2 = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.4,
    rad_seq_end=0.1,
    min_allowed_rad=2,
    n_rads=90,
)
rcp2.run(1)

In [ ]:
rcp2.rad_seq_start=0.4
rcp2.rad_seq_end=0.1
rcp2.n_rads=80
rcp2.run(3, progress_bar=True)
rcp2.unfilled_circles

In [ ]:
# init
rcp3 = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.4,
    rad_seq_end=0.1,
    min_allowed_rad=2,
    n_rads=90,
)
rcp3.run(1)

In [ ]:
rcp3.rad_seq_start=0.4
rcp3.rad_seq_end=0.1
rcp3.n_rads=80
rcp3.run(5, progress_bar=True)
rcp3.unfilled_circles

In [ ]:
d1 = rcp.unfilled_circles.intersection(rcp2.unfilled_circles).buffer(4, cap_style=2).difference(rcp3.unfilled_boundary.buffer(3))

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-1.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
stp.angles = np.linspace(np.radians(720), 0,  stp.d_buffers.shape[0])
fills = []
for p in d1:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

layer1 = gp.merge_LineStrings(fills)

In [ ]:
morse_lss = []
for ls in tqdm(layer1):
    morsed = ls.buffer(1e-7).buffer(-1e-7).boundary
    morse_lss.append(morsed)

In [ ]:
filt_lss = [ls for ls in morse_lss if ls.length > 0.1]

In [ ]:
layer2 = gp.merge_LineStrings(filt_lss)

In [ ]:
layer3 = gp.merge_LineStrings([ls for ls in layer2 if ls.length > 0.5])

In [ ]:
sns.displot([np.log10(ls.length) for ls in filt_lss])

In [ ]:
sns.displot([np.log10(ls.length) for ls in layer3])

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
# sk.stroke(1)
# sk.geometry(layer1)
sk.stroke(2)
sk.geometry(layer3)
sk.display(color_mode='none')

In [ ]:
from time import sleep

In [ ]:
pbar = tqdm(total = 76)
for i in range(31):
    sleep(0.1)
    pbar.update()

In [ ]:
5.256e5

In [ ]:
1/525600.

In [ ]:
spy = 1/(525600 * 60)


In [ ]:
import pandas as pd

In [ ]:
td = pd.to_timedelta(secs_remaining, unit='S')

In [ ]:
td += pd.to_timedelta(127, unit='m')

In [ ]:
from string import Template

class DeltaTemplate(Template):
    delimiter = "%"

def strfdelta(tdelta, fmt):
    
    days = tdelta.days
    d = {}
    d['Y'], rem = divmod(days, 365)
    d['D'] = rem
    
    hours, rem = divmod(tdelta.seconds, 3600)
    minutes, seconds = divmod(rem, 60)
    
    d["H"] = '{:02d}'.format(hours)
    d["M"] = '{:02d}'.format(minutes)
    d["S"] = '{:02d}'.format(seconds)
    t = DeltaTemplate(fmt)
    return t.substitute(**d)

In [ ]:
strfdelta(td,'%Y:%H:%M:%S')

In [ ]:
td.isoformat()

In [ ]:
sr = pd.to_timedelta(secs_remaining, unit='S')
strfdelta(sr,'%Y:%D:%H:%M:%S')

In [ ]:
def iso8601(tdelta):
    ts = tdelta.total_seconds()
    d = int(ts // 86400)
    s = round(ts % 60, 6)
    hms = int(ts // 3600 % 24), int(ts // 60 % 60), s if s % 1 != 0 else int(s)
    t = ''.join([str(p[0]) + p[1] for p in zip(hms, ['H', 'M', 'S']) if p[0]])
    sep = 'T' if any(hms) else ''
    return 'P' + (str(d) + 'D' if d else '') + sep + (t if ts else 'T0S')

In [ ]:
iso8601(sr)

In [ ]:
se = pd.to_timedelta(secs_elapsed, unit='S')
strfdelta(se,'%Y:%D:%H:%M:%S')

In [ ]:
secs_elapsed = 31 * spy**-1 + 7623

In [ ]:
secs_elapsed

In [ ]:
total_secs = (76 * spy**-1) + 812399

In [ ]:
total_secs

In [ ]:
secs_remaining = total_secs - secs_elapsed

In [ ]:
secs_remaining

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0121_morsed_circle_pack_diffs.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.2mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

# Try 6

In [ ]:
page_x_inches: float = 14. # inches
page_y_inches: float = 11 # inches
border:float = 30.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.48,
    rad_seq_end=0.05,
    min_allowed_rad=2,
    n_rads=30,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.6
rcp.rad_seq_end=0.04
rcp.n_rads=80
rcp.run(50, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(2.1, 2.8).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
occluded_tris = tris.intersection(rcp.unfilled_circles).buffer(-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.4, 2.2).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris]).buffer(-1e-6)

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(10), rad_seq_start=0.15,
    rad_seq_end=0.15, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.2
trcp.rad_seq_end=0.2
trcp.run(4)

In [ ]:
length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.5, 1.8).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = gp.merge_Polygons([all_polys, occluded_tris])

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(1)
sk.geometry(layer1)
sk.stroke(2)
sk.geometry(layer2)
sk.stroke(3)
sk.geometry(layer3)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0274_cpack_beams.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.1mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

# Try 7

In [ ]:
page_x_inches: float = 7. # inches
page_y_inches: float = 5 # inches
border:float = 15.

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds


In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.48,
    rad_seq_end=0.05,
    min_allowed_rad=2,
    n_rads=30,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.6
rcp.rad_seq_end=0.04
rcp.n_rads=80
rcp.run(50, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(2.1, 2.8).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
occluded_tris = tris.intersection(rcp.unfilled_circles).buffer(-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.4, 2.2).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris]).buffer(-1e-6)

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(10), rad_seq_start=0.15,
    rad_seq_end=0.15, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.2
trcp.rad_seq_end=0.2
trcp.run(4)

In [ ]:
length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.5, 1.8).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = gp.merge_Polygons([all_polys, occluded_tris])

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)

sk.scale('1mm')
sk.stroke(1)
sk.geometry(layer1)
sk.stroke(2)
sk.geometry(layer2)
sk.stroke(3)
sk.geometry(layer3)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0278_cpack_beams.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.1mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)